In [2]:
# ====================================================
# Character Recognition with DenseNet121 (Transfer Learning, TF/Keras)
# ====================================================

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from collections import defaultdict

# -----------------------
# CONFIG
# -----------------------
train_dir = "/kaggle/input/ocr-data/OCR_data/train_data"  
test_dir  = "/kaggle/input/ocr-data/OCR_data/test_data"   
img_size = (128, 128)
batch_size = 16
epochs = 50 

# -----------------------
# CLASS MAPPING
# -----------------------
train_class_names = sorted({f[0] for f in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, f))})
print("Train classes (first letters):", train_class_names)

class_to_index = {cls: idx for idx, cls in enumerate(train_class_names)}
index_to_class = {idx: cls for cls, idx in class_to_index.items()}
num_classes = len(class_to_index)

print("Class mapping:", class_to_index)
print("Number of classes:", num_classes)

# -----------------------
# DATASET LOADER
# -----------------------
keys_tensor = tf.constant(list(class_to_index.keys()))
vals_tensor = tf.constant(list(class_to_index.values()), dtype=tf.int32)
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor),
    default_value=-1
)

def process_path(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=1)   # grayscale
    img = tf.image.resize(img, img_size)
    img = (tf.cast(img, tf.float32) / 127.5) - 1.0

    parts = tf.strings.split(path, os.sep)
    folder_name = parts[-2]
    label_char = tf.strings.substr(folder_name, 0, 1)
    label = table.lookup(label_char)

    return img, label

train_files = tf.data.Dataset.list_files(train_dir + "/*/*", shuffle=True)
test_files  = tf.data.Dataset.list_files(test_dir + "/*/*", shuffle=False)

full_train_ds = train_files.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

# Split into train/validation (80/20)
train_size = int(0.8 * len(list(train_files)))
train_ds = (full_train_ds.take(train_size)
            .batch(batch_size)
            .prefetch(tf.data.AUTOTUNE))
val_ds = (full_train_ds.skip(train_size)
          .batch(batch_size)
          .prefetch(tf.data.AUTOTUNE))

test_ds = (test_files
           .map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
           .batch(batch_size)
           .prefetch(tf.data.AUTOTUNE))

# -----------------------
# MODEL: DenseNet121 Backbone
# -----------------------
densenet_base = DenseNet121(
    input_shape=(128,128,3),
    include_top=False,
    weights="imagenet"
)

densenet_base.trainable = False  # freeze backbone

inputs = layers.Input(shape=(128,128,1))
x = layers.Concatenate()([inputs, inputs, inputs])   # grayscale → RGB
x = densenet_base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax",
                       kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)

model = models.Model(inputs, outputs)
model.summary()

Train classes (first letters): ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Class mapping: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15, 'G': 16, 'H': 17, 'I': 18, 'J': 19, 'K': 20, 'L': 21, 'M': 22, 'N': 23, 'O': 24, 'P': 25, 'Q': 26, 'R': 27, 'S': 28, 'T': 29, 'U': 30, 'V': 31, 'W': 32, 'X': 33, 'Y': 34, 'Z': 35, 'a': 36, 'b': 37, 'c': 38, 'd': 39, 'e': 40, 'f': 41, 'g': 42, 'h': 43, 'i': 44, 'j': 45, 'k': 46, 'l': 47, 'm': 48, 'n': 49, 'o': 50, 'p': 51, 'q': 52, 'r': 53, 's': 54, 't': 55, 'u': 56, 'v': 57, 'w': 58, 'x': 59, 'y': 60, 'z': 61}
Number of classes: 62


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 128, 128,  │          0 │ input_layer_3[0]… │
│ (Concatenate)       │ 3)                │            │ input_layer_3[0]… │
│                     │                   │            │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ densenet121         │ (None, 4, 4,      │  7,037,504 │ concatenate_1[0]… │
│ (Functional)        │ 1024)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1024)      │          0 │ densenet121[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1024)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 62)        │     63,550 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,101,054 (27.09 MB)

 Trainable params: 63,550 (248.24 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [3]:
# -----------------------
# COMPILE & TRAIN
# -----------------------
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=[early_stop]
)

Epoch 1/50


I0000 00:00:1757893537.152799      95 service.cc:148] XLA service 0x7bcdf8003cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757893537.153713      95 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1757893537.153737      95 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1757893539.843002      95 cuda_dnn.cc:529] Loaded cuDNN version 90300


 5/31 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0431 - loss: 6.6983     

I0000 00:00:1757893548.530899      95 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


31/31 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.0392 - loss: 6.2414 - val_accuracy: 0.0323 - val_loss: 4.0166
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.0441 - loss: 5.1555 - val_accuracy: 0.1290 - val_loss: 3.5147
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.0562 - loss: 4.3108 - val_accuracy: 0.2177 - val_loss: 3.0556
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.1111 - loss: 3.9372 - val_accuracy: 0.4435 - val_loss: 2.5163
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.1871 - loss: 3.3667 - val_accuracy: 0.4677 - val_loss: 2.3179
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.2043 - loss: 3.1079 - val_accuracy: 0.5887 - val_loss: 1.8784
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2418 - loss: 2.9412 - val_accuracy: 0.7500 - val_loss: 1.6847
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.3255 - loss: 2.4298 - val_accuracy: 0.7419 - val_loss: 1.5

In [4]:
# -----------------------
# SAVE & LOAD
# -----------------------
model.save("densenet121_char_tf.h5")
print("Model saved!")

loaded_model = tf.keras.models.load_model("densenet121_char_tf.h5")
print("Model loaded!")

Model saved!
Model loaded!


In [5]:
# -----------------------
# FINAL EVALUATION (TEST DATA)
# -----------------------
correct = 0
wrong = 0
total = 0

y_true = []
y_pred = []
class_correct = defaultdict(int)
class_total = defaultdict(int)

for images, labels in test_ds:
    preds = loaded_model.predict(images, verbose=0)
    predicted_classes = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(predicted_classes)

    correct += np.sum(predicted_classes == labels.numpy())
    wrong += np.sum(predicted_classes != labels.numpy())
    total += labels.shape[0]

    for true, pred in zip(labels.numpy(), predicted_classes):
        class_total[true] += 1
        if true == pred:
            class_correct[true] += 1

print("\n=== Per-Class Results ===")
for idx in sorted(class_total.keys()):
    total_i = class_total[idx]
    correct_i = class_correct[idx]
    wrong_i = total_i - correct_i
    print(f"Class {index_to_class[idx]}: Correct={correct_i}, Wrong={wrong_i}, Total={total_i}, Acc={100*correct_i/total_i:.2f}%")

print("\n=== Overall Results ===")
print(f"Correct predictions: {correct}")
print(f"Wrong predictions: {wrong}")
print(f"Total images: {total}")
print(f"Accuracy: {100.0 * correct / total:.2f}%")

precision = precision_score(y_true, y_pred, average="macro")
recall = recall_score(y_true, y_pred, average="macro")
f1 = f1_score(y_true, y_pred, average="macro")

print("\n=== Precision / Recall / F1 (Macro) ===")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=[index_to_class[i] for i in range(num_classes)]))


=== Per-Class Results ===
Class 0: Correct=10, Wrong=15, Total=25, Acc=40.00%
Class 1: Correct=3, Wrong=22, Total=25, Acc=12.00%
Class 2: Correct=6, Wrong=19, Total=25, Acc=24.00%
Class 3: Correct=17, Wrong=8, Total=25, Acc=68.00%
Class 4: Correct=8, Wrong=17, Total=25, Acc=32.00%
Class 5: Correct=10, Wrong=15, Total=25, Acc=40.00%
Class 6: Correct=13, Wrong=12, Total=25, Acc=52.00%
Class 7: Correct=2, Wrong=23, Total=25, Acc=8.00%
Class 8: Correct=8, Wrong=17, Total=25, Acc=32.00%
Class 9: Correct=6, Wrong=19, Total=25, Acc=24.00%
Class A: Correct=13, Wrong=12, Total=25, Acc=52.00%
Class B: Correct=22, Wrong=3, Total=25, Acc=88.00%
Class C: Correct=16, Wrong=9, Total=25, Acc=64.00%
Class D: Correct=20, Wrong=5, Total=25, Acc=80.00%
Class E: Correct=9, Wrong=16, Total=25, Acc=36.00%
Class F: Correct=10, Wrong=15, Total=25, Acc=40.00%
Class G: Correct=12, Wrong=13, Total=25, Acc=48.00%
Class H: Correct=19, Wrong=6, Total=25, Acc=76.00%
Class I: Correct=15, Wrong=10, Total=25, Acc=60.00